In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
from tqdm import tqdm_notebook
import IPython
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['load']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
df_train = pd.read_csv('data/new_df_train.csv', index_col='id')
# df_test = pd.read_csv('data/test_clear_items_heros_demage_data.csv', index_col='id')
items = pd.read_csv('data/dota2_items.csv',index_col="item_id")
abil = pd.read_csv('data/dota2_abilities.csv',index_col="ability_id")
heroes = pd.read_csv('data/dota2_heroes.csv',index_col="hero_id")

items["cost"].fillna(0,inplace=True)
items["qual"].fillna("none",inplace=True)


In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## NEW DATA REPRAPERING FROM JSON

In [8]:
import json
import tqdm


def load(path='data/dota2_skill_train.jsonlines'):
    e=0
    dataframe=[]
    with open(path) as fin:
        for line in tqdm.tqdm_notebook(fin,total = 100000):
            record = json.loads(line)
            rd={}
            damage_targets={}
            ability_upgrades={}
            item_purchase_log={}
            hero_roles={}
            radiant_heroes={}
            dire_heroes={}
            

            #damage parsing
            for t in record["damage_targets"]:
                if(t in damage_targets):
                    damage_targets[t.split('_')[2]]+=record["damage_targets"][t]
                else:
                    damage_targets[t.split('_')[2]]=record["damage_targets"][t]
            #ability_upgrades        
            for a in record["ability_upgrades"]:
                try:
                    bs=json.loads(abil.loc[a,"behavior"].replace("'",'"').replace("None",'"None"'))
                    l=len(bs)
                    for b in bs:
                        if b in ability_upgrades:
                            ability_upgrades[b]+=1/l
                        else:
                            ability_upgrades[b]=1/l
                except:
                    ability_upgrades[ abil.loc[a,"behavior"] ]=1
            #item_purchase_log
            for i in record["item_purchase_log"]:
                q=items.loc[i["item_id"]]
                if q["qual"] in item_purchase_log:
                    item_purchase_log[q["qual"] ]+=1
                    item_purchase_log[q["qual"]+"_cost"]+=q["cost"]

                else:
                    item_purchase_log[q["qual"] ]=1
                    item_purchase_log[q["qual"]+"_cost"] = q["cost"]
            #hero info
            hero=heroes.loc[record["hero_id"]]
            
            
            for rol in json.loads(hero["roles"].replace("'",'"')):
                hero_roles[rol]=1
            #dire hero info
            for he in record["dire_heroes"]:
                hero=heroes.loc[he]
                for d in json.loads(hero["roles"].replace("'",'"')):
                    if d in dire_heroes:
                        dire_heroes[d]+=1
                    else:
                        dire_heroes[d]=1
            #dire radian info
            for he in record["radiant_heroes"]:
                hero=heroes.loc[he]
                for d in json.loads(hero["roles"].replace("'",'"')):
                    if d in radiant_heroes:
                        radiant_heroes[d]+=1
                    else:
                        radiant_heroes[d]=1
                        
                        
            #SBORKA 
            for i in damage_targets:
                rd["damage_"+i]=damage_targets[i]
            for i in ability_upgrades:
                rd["abil_"+i]=ability_upgrades[i]
            for i in item_purchase_log:
                rd["itm_"+str(i)]=item_purchase_log[i]
            for i in hero_roles:
                rd["heroes_"+i]=hero_roles[i]
            #other heroes
            for i in dire_heroes:
                rd["dire_heroes"+i]=dire_heroes[i]
            for i in radiant_heroes:
                rd["radiant_heroes"+i]=radiant_heroes[i]
            #all other parametr
            for r in record:
                if isinstance(record[r],int) or isinstance(record[r],float) or isinstance(record[r],str):
                    rd[r]=record[r]
            for h in heroes:
                if isinstance(heroes[h],int) or isinstance(heroes[h],float):
                    rd[h]=heroes[h]

            
            
                
            dataframe.append(rd)   
            e+=1
    return dataframe


NameError: name 'IPython' is not defined

In [11]:
IPython.load_extensions('pikle/show.js')

AttributeError: module 'IPython' has no attribute 'load_extensions'

In [ ]:
df=load()

In [ ]:
df_train=pd.DataFrame.from_dict(df).set_index("id").fillna(0)
df_train.to_csv('data/new_df_train.csv')
df_train.head()

In [ ]:
abil.head()

In [ ]:
df_train=pd.DataFrame()
for r in record:
    df_train[r]=0

In [ ]:
df_train

**heroes data preporation**

heroes.head()

**Items category+ data**

**Data transform**

**Team mean heroes parametrs**

**Damage targets params** 

<h3 style="color:red">Final items cost</h3>

**FULL ITEMS FROM dota.com**

**NEW REGRESSIND PARAMS**

In [ ]:
np.array(df_train.columns)

## _Test_

In [ ]:
maches=[]

In [ ]:
import requests
import tqdm
key="F8409D2B7A7D3E13A59761D534ECA044"
conf={
    "key":key,
    "start_at_match_id":5634189078,
    "skill":3
}

In [ ]:
le=0
l2=0
was=[]
while(le<20000 and l2<20000):
    r = requests.get(url="https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/",params=conf)
    for i in r.json()["result"]["matches"]:
        maches.append(i["match_id"])
    if(len(set(maches))==l2):
        print(maches[-1]-1)
        conf["start_at_match_id"]=5634189078
        conf2={
            "key":key,
            "match_id":0,
        }
        with open("skill3.jsonlines","a") as fout:
            for m in tqdm.tqdm_notebook(maches):
                if(m in was):
                    break
                conf["match_id"]=m
                r = requests.get(url="https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001/",params=conf2)
                print(r.json(),file=fout)
        le+=len(maches)
        l2=0
        was=was+maches
        maches=[]
        time.sleep(5)
        continue
        
    conf["start_at_match_id"]=maches[-1]-1
    l2=len(set(maches))
 

In [ ]:
len(set(maches) )

In [ ]:
m2=maches

In [ ]:
conf={
    "key":key,
    "match_id":0,
}
with open("skill3.jsonlines","a") as fout:
    for m in tqdm.tqdm_notebook(maches):
        conf["match_id"]=m
        r = requests.get(url="https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001/",params=conf)
        print(r.json(),file=fout)


In [ ]:
fout.close()

In [ ]:
import json
import tqdm
# df_train["demage_len"]=0
# with open('data/dota2_skill_train.jsonlines',total=100000) as fin:
#     for line in tqdm.tqdm_notebook(fin):
#         record = json.loads(line)
#         df_train.loc[record['id'],'demage_len']=len(record["damage_targets"])
n=0
df_test["demage_len"]=0
with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin,total=40000):
        record = json.loads(line)
        print(len(set([r for r in record["damage_targets"]]))-len([r for r in record["damage_targets"]]))
        n+=1
        if(n>15):
            break
#         df_test.loc[record['id'],'demage_len']=len(record["damage_targets"])

In [ ]:
record["denies"]

In [ ]:
[r for r in record]

In [ ]:
df_train.to_csv('data/train_clear_items_heros_demage_dl_data.csv')
df_test.to_csv('data/test_clear_items_heros_demage_dl_data.csv')

In [ ]:
items = pd.read_csv('data/dota2_items.csv',index_col="item_id")
abil = pd.read_csv('data/dota2_abilities.csv',index_col="ability_id")
her = pd.read_csv('data/dota2_heroes.csv',index_col="hero_id")

In [ ]:
her.head()

In [ ]:
for r in record:
    if not isinstance(record[r],int) and not isinstance(record[r],float) and not isinstance(record[r],str):
        print(r)

In [ ]:
record["series"]["player_gold"][-1],record["gold_spent"]

In [ ]:
record["gold_per_min"]*record["duration"]/60

In [ ]:
for r in record:
        print(r)

**Mean speed of level UP**

In [ ]:
n=8
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin,total=99871):
        record = json.loads(line)
        n-=1
        if(n<1):
            break

In [ ]:
import matplotlib.pyplot as plt
levels=[i for i in range(len(record["ability_upgrades"]))]
times=[ t["time"] for t in record["ability_upgrades"] ]
plot(levels,times)
# plot(record['series']['time'],record['series']['dire_gold'])
# plot(record['series']['time'],record['series']['radiant_gold'])


xlabel('n items')
ylabel('time')
gcf().autofmt_xdate(0,45)
grid(True)
show()
print(record["winner_team"],record["skilled"])

In [ ]:
for r in record:
    print(r)

In [ ]:
df_hero["attack_type"] = LabelEncoder().fit_transform(df_hero["attack_type"])
df_hero["primary_attr"] = LabelEncoder().fit_transform(df_hero["primary_attr"])

In [ ]:
df_hero.head()

In [ ]:
df_train=df_train.join(df_hero, lsuffix='hero_id', rsuffix='hero_id')

In [ ]:
df_train['hero_id'].head()

**Data encoding**

In [ ]:
df_train = pd.read_csv('data/train_clear_items_heros_demage_data.csv', index_col='id')
df_test = pd.read_csv('data/test_clear_items_heros_demage_data.csv', index_col='id')

In [ ]:
def drop_flud(df):
    df=df.drop(["base_health","base_mana","first_blood_claimed","pre_game_duration","base_mr"])
    

In [ ]:
for c in df_train.columns:
    print(c)

In [ ]:
for c in df_train.columns:
    l=len(set(df_train[c]))
    if l<25 and l>2 and df_train[c].dtypes=="int64":
        print(c)

In [ ]:
df_train["camps_stacked"].head(10)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lable_enc_list=['player_team','winner_team']
for lb in lable_enc_list:
    df_train[lb] = LabelEncoder().fit_transform(df_train[lb])


from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
lable_ohe_list=['player_team',"attack_rate"]
for lb in lable_ohe_list:
    f=enc.fit(df_train[lb].values.reshape(-1,1))
    df_train[lb]=enc.transform(df_train[lb].values.reshape(-1,1)).toarray()

In [ ]:
df_train.head()